# Pets Breeds Classification

## Import TensorFlow and other libraries

<b>Versions used:</b> <br>
*Python* - 3.7.0 <br>
*Tensorflow* - 2.3.0 <br>
*Keras* - 2.4.0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import PIL
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications import inception_v3

## Download the dataset

The datased used in this project, [The Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/), contains photos of 37 different breeds of cats and dogs with roughly 200 images for each class.

In [ ]:
import pathlib

dataset_url = "https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz"
data_dir = tf.keras.utils.get_file('images', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

The downloaded dataset should contain 7,390 images:

In [ ]:
image_count = len(list(data_dir.glob('*.jpg'))) + len(list(data_dir.glob('*/*.jpg')))
print(str(image_count) + " images successfully downloaded.")

Example of an image in the dataset:

In [ ]:
images = list(data_dir.glob('*.jpg')) + list(data_dir.glob('*/*.jpg'))
PIL.Image.open(str(images[500]))

## Load dataset to a TensorFlow dataset object

So far the dataset is just a set of images in a directory. In order to train a model, a *tf.data.Dataset* file have to be created. 

First adapt directory hierarchy to fit keras  *image_dataset_from_directory* requirements. Create subdirectories for each breed and assign images to appropriate class-directory.

In [ ]:
import os

breeds = []
        
for file in data_dir.glob("*"):
    if file.suffix == '.jpg':
        last_floor_pos = file.name.rfind('_')
        breed = file.name[:last_floor_pos].lower()
        img_index = file.name[last_floor_pos + 1:]
        
        if not data_dir.joinpath(breed).is_dir():
            data_dir.joinpath(breed).mkdir()
            
        file.replace(data_dir.joinpath(breed, img_index))
        
    elif not file.is_dir():
        os.remove(file) 
        
for file in data_dir.glob("*"):
    breeds.append(file.name)

Define loader parameters.

In [ ]:
img_height = 224
img_width = 224
split = 0.2       # 80% of the images will be used for training and 20% for validation

Prepare a training data sample.

In [ ]:
train_img_count = math.ceil(image_count * (1 - split))

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=split,
    subset='training',
    seed=2021,
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=train_img_count)

Prepare a validation data sample.

In [ ]:
val_img_count = image_count - train_img_count

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=split,
    subset="validation",
    seed=2021,
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=val_img_count)

## Configure the dataset

Add buffered prefetching to ensure the data can be yield from disk without having I/O become blocking.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Standarise the data to use [0, 1] range, instead of [0, 255] typical for the RGB channels.

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

train_images, train_labels = next(iter(normalized_train_ds))
val_images, val_labels = next(iter(normalized_val_ds))

Example of an image in standarised dataset:

In [ ]:
image = train_images[33]
plt.imshow(image)
plt.show()

## Create the model

Create data augmentation layer to decrease risk of overfitting.

In [ ]:
input_shape = (img_height, img_width, 3)

data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=input_shape),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
])

Example of data augmentation:

In [ ]:
image_array = keras.preprocessing.image.img_to_array(image)
image_tensor = tf.expand_dims(image_array, 0)

plt.figure(figsize=(10, 10))

for i in range(9):
    augmented_image = data_augmentation(image_tensor)
    plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")

Load ImageNet pretrained model to implement transfer learning. Use `include_top=False` parameter to remove the last predicting layer of the pretrained model and replace them with own predicting layers. Freeze the weights of the model by setting `trainable=False` of each component layer.

In [ ]:
imagenet=inception_v3.InceptionV3(weights='imagenet', include_top=False)
imagenet.summary()

for layer in imagenet.layers:
    layer.trainable=False

Define final model layers.

In [ ]:
model = keras.Sequential([
    data_augmentation,
    imagenet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(37, kernel_initializer='uniform', activation="softmax")
])

## Compile the model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.summary()

## Train the model

In [ ]:
history = model.fit(train_images,
                    train_labels,
                    epochs=10,
                    validation_data=(val_images, val_labels),
                    verbose=1)

## Test the trained model

First introduce a helper function to predict a breed.

In [ ]:
def predict_breed(image_path, best_n = 1):

    img = keras.preprocessing.image.load_img(image_path,
        target_size=(img_height, img_width))
    
    img_array = keras.preprocessing.image.img_to_array(img) / 255
    img_array = tf.expand_dims(img_array, 0)

    prediction_probabilities = model.predict(img_array)
    top_breeds_indexes = prediction_probabilities[0].argsort()[-best_n:]
    
    top_predictions = []
    for index in top_breeds_indexes[::-1]:
        top_predictions.append((breeds[index], prediction_probabilities[0][index]))
        
    plt.imshow(img)
    plt.show()
    return top_predictions

Let's try a photo of a *german shorthaired*:

In [ ]:
dog_image_url = \
    'https://media.nextechclassifieds.com/img/listings/bl/bluelinegundogs/listing_pic_1580328_1580345588.jpeg'
dog_image_path = keras.utils.get_file('german_shorthaired',
        origin=dog_image_url)


predict_breed(dog_image_path, 5)

Now a *birman* photo:

In [ ]:
cat_image_url = \
    'https://birman.eu/cms/core_files/thumbs/1500x1100/1.DSC_0344.jpg'
cat_image_path = keras.utils.get_file('birman',
        origin=cat_image_url)


predict_breed(cat_image_path, 5)

# Confusion Matrix

Plot the confusion matrix on the validation dataset.

In [ ]:
model_predictions = model.predict(val_images)

In [ ]:
y_true = [np.argmax(row) for row in val_labels]
y_pred = [np.argmax(row) for row in model_predictions]

matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 10))
plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()

tick_marks = np.arange(len(breeds))
plt.xticks(tick_marks, breeds, rotation=90)
plt.yticks(tick_marks, breeds)

plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.tight_layout()

# Save the model

In [ ]:
model.save(f"model/breeds_classification")